In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv('../../data/processed/LOLOracleDataWr.csv')

In [3]:
df.head()

,BTop,BJng,BMid,BAdc,BSup,RTop,RJng,RMid,RAdc,RSup,Winner
0,0.524364,0.506967,0.497972,0.523989,0.438144,0.533592,0.494477,0.491465,0.503883,0.492260,1
1,0.524364,0.504318,0.565594,0.523989,0.491429,0.516260,0.509972,0.559211,0.504505,0.440678,0
2,0.513399,0.486450,0.553073,0.525246,0.524819,0.489831,0.539535,0.465315,0.470158,0.527418,0
3,0.511305,0.505097,0.502868,0.524169,0.534702,0.508671,0.502106,0.508434,0.393443,0.492582,1
4,0.524364,0.509764,0.421169,0.525246,0.516461,0.499863,0.502106,0.445918,0.465315,0.473513,1


In [4]:
df.columns

Index(['BTop', 'BJng', 'BMid', 'BAdc', 'BSup', 'RTop', 'RJng', 'RMid', 'RAdc',
       'RSup', 'Winner'],
      dtype='object')

In [5]:
Red = ['RTop', 'RJng', 'RMid', 'RAdc','RSup']
Blue = ['BTop', 'BJng', 'BMid', 'BAdc', 'BSup']

In [6]:
for i in Red:
    df[i]=-df[i]

In [7]:
df.head()

,BTop,BJng,BMid,BAdc,BSup,RTop,RJng,RMid,RAdc,RSup,Winner
0,0.524364,0.506967,0.497972,0.523989,0.438144,-0.533592,-0.494477,-0.491465,-0.503883,-0.492260,1
1,0.524364,0.504318,0.565594,0.523989,0.491429,-0.516260,-0.509972,-0.559211,-0.504505,-0.440678,0
2,0.513399,0.486450,0.553073,0.525246,0.524819,-0.489831,-0.539535,-0.465315,-0.470158,-0.527418,0
3,0.511305,0.505097,0.502868,0.524169,0.534702,-0.508671,-0.502106,-0.508434,-0.393443,-0.492582,1
4,0.524364,0.509764,0.421169,0.525246,0.516461,-0.499863,-0.502106,-0.445918,-0.465315,-0.473513,1


In [8]:
#For all entries with Blue = 1 and Red = -1

for i in range(len(df)):
    if df.loc[i,'Winner']==0:
        df.loc[i,'Winner']=1
    else:
        df.loc[i,'Winner']=-1
        
df['Winner'] = df['Winner'].astype(int)

In [9]:
df.head()

,BTop,BJng,BMid,BAdc,BSup,RTop,RJng,RMid,RAdc,RSup,Winner
0,0.524364,0.506967,0.497972,0.523989,0.438144,-0.533592,-0.494477,-0.491465,-0.503883,-0.492260,-1
1,0.524364,0.504318,0.565594,0.523989,0.491429,-0.516260,-0.509972,-0.559211,-0.504505,-0.440678,1
2,0.513399,0.486450,0.553073,0.525246,0.524819,-0.489831,-0.539535,-0.465315,-0.470158,-0.527418,1
3,0.511305,0.505097,0.502868,0.524169,0.534702,-0.508671,-0.502106,-0.508434,-0.393443,-0.492582,-1
4,0.524364,0.509764,0.421169,0.525246,0.516461,-0.499863,-0.502106,-0.445918,-0.465315,-0.473513,-1


#### Lets try training the data without encoding every single champion using one hot enocder / dummies

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X = df.drop('Winner',axis=1)
y = df['Winner']

In [12]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=101)

In [13]:
from sklearn.svm import SVC

In [14]:
svm = SVC()

In [15]:
svm.fit(X_train,y_train)

/Users/deepakbuddha/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [16]:
predictions = svm.predict(X_test)

In [17]:
from sklearn.metrics import classification_report,confusion_matrix

In [18]:
print(confusion_matrix(y_test,predictions))
print('\n')
print(classification_report(y_test,predictions))

[[1010 1939]
 [ 711 2244]]


              precision    recall  f1-score   support

          -1       0.59      0.34      0.43      2949
           1       0.54      0.76      0.63      2955

    accuracy                           0.55      5904
   macro avg       0.56      0.55      0.53      5904
weighted avg       0.56      0.55      0.53      5904



### Lets try using Gridsearch to get the optimum values for hyperparameters C and Gamma

In [19]:
from sklearn.model_selection import GridSearchCV

In [20]:
params = {'C':[0.1,1,10,100,1000],'gamma':[1,0.1,0.01,0.001,0.0001]}

In [21]:
grid = GridSearchCV(SVC(),params,verbose=3)

In [22]:
grid.fit(X_train,y_train)

/Users/deepakbuddha/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 25 candidates, totalling 75 fits
[CV] C=0.1, gamma=1 ..................................................
[CV] ...................... C=0.1, gamma=1, score=0.536, total=   3.6s
[CV] C=0.1, gamma=1 ..................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.6s remaining:    0.0s


[CV] ...................... C=0.1, gamma=1, score=0.536, total=   3.6s
[CV] C=0.1, gamma=1 ..................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    7.2s remaining:    0.0s


[CV] ...................... C=0.1, gamma=1, score=0.532, total=   3.5s
[CV] C=0.1, gamma=0.1 ................................................
[CV] .................... C=0.1, gamma=0.1, score=0.506, total=   3.6s
[CV] C=0.1, gamma=0.1 ................................................
[CV] .................... C=0.1, gamma=0.1, score=0.506, total=   3.7s
[CV] C=0.1, gamma=0.1 ................................................
[CV] .................... C=0.1, gamma=0.1, score=0.506, total=   3.6s
[CV] C=0.1, gamma=0.01 ...............................................
[CV] ................... C=0.1, gamma=0.01, score=0.506, total=   3.6s
[CV] C=0.1, gamma=0.01 ...............................................
[CV] ................... C=0.1, gamma=0.01, score=0.506, total=   3.6s
[CV] C=0.1, gamma=0.01 ...............................................
[CV] ................... C=0.1, gamma=0.01, score=0.506, total=   3.5s
[CV] C=0.1, gamma=0.001 ..............................................
[CV] .

[CV] ..................... C=1000, gamma=1, score=0.572, total=   6.1s
[CV] C=1000, gamma=1 .................................................
[CV] ..................... C=1000, gamma=1, score=0.569, total=   6.1s
[CV] C=1000, gamma=1 .................................................
[CV] ..................... C=1000, gamma=1, score=0.566, total=   6.4s
[CV] C=1000, gamma=0.1 ...............................................
[CV] ................... C=1000, gamma=0.1, score=0.567, total=   3.7s
[CV] C=1000, gamma=0.1 ...............................................
[CV] ................... C=1000, gamma=0.1, score=0.572, total=   3.9s
[CV] C=1000, gamma=0.1 ...............................................
[CV] ................... C=1000, gamma=0.1, score=0.573, total=   3.7s
[CV] C=1000, gamma=0.01 ..............................................
[CV] .................. C=1000, gamma=0.01, score=0.568, total=   3.7s
[CV] C=1000, gamma=0.01 ..............................................
[CV] .

[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed:  4.7min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [23]:
grid_predictions = grid.predict(X_test)

In [24]:
print(confusion_matrix(y_test,grid_predictions))
print('\n')
print(classification_report(y_test,grid_predictions))

[[1504 1445]
 [1157 1798]]


              precision    recall  f1-score   support

          -1       0.57      0.51      0.54      2949
           1       0.55      0.61      0.58      2955

    accuracy                           0.56      5904
   macro avg       0.56      0.56      0.56      5904
weighted avg       0.56      0.56      0.56      5904

